# Neural Language Model
- 목적 : 
- Output : 뒤의 단어 예측
- Model : FCN
- Loss : Cross Entropy

##### 0) 라이브러리 불러오기

In [222]:
# (1) 딥러닝 라이브러리
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

##### 1) 데이터 불러오기

(1) 변수 초기화

In [223]:
NGRAM_SIZE = 2
EMBEDDING_DIM = 10

(2) 데이터 불러오기 (인터넷 데이터)
- https://www.unesco.org/en/weeks/world-novel

In [224]:
sentence_train = """
Suspense novels follow a plot that induces a unsettling sense of worry or anticipation in readers.
This is done so by letting them in on details that the story's protagonist is often not weary of.
Often, mystery or thriller novels fall under the suspense category and these three genre titles are also used interchangeably because of the blur lines.
But the distinguishing factor between these three lies in the type of suspense that the story follows.
In mystery novels, we follow the journey of the protagonist who is would be trying to solve a crime.
In thriller novels, we see the central characters or main protagonist being suspended in ever-present danger and risk.
And in suspense novels, the plot takes its readers through an unsettling sense of worry and anticipation in readers by revealing potentially dangerous details that the very protagonist in the story could not be aware of.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.
# Each tuple is ([ word_i-NGRAM_SIZE, ..., word_i-1 ], target word)

In [225]:
sentence_train

['Suspense',
 'novels',
 'follow',
 'a',
 'plot',
 'that',
 'induces',
 'a',
 'unsettling',
 'sense',
 'of',
 'worry',
 'or',
 'anticipation',
 'in',
 'readers.',
 'This',
 'is',
 'done',
 'so',
 'by',
 'letting',
 'them',
 'in',
 'on',
 'details',
 'that',
 'the',
 "story's",
 'protagonist',
 'is',
 'often',
 'not',
 'weary',
 'of.',
 'Often,',
 'mystery',
 'or',
 'thriller',
 'novels',
 'fall',
 'under',
 'the',
 'suspense',
 'category',
 'and',
 'these',
 'three',
 'genre',
 'titles',
 'are',
 'also',
 'used',
 'interchangeably',
 'because',
 'of',
 'the',
 'blur',
 'lines.',
 'But',
 'the',
 'distinguishing',
 'factor',
 'between',
 'these',
 'three',
 'lies',
 'in',
 'the',
 'type',
 'of',
 'suspense',
 'that',
 'the',
 'story',
 'follows.',
 'In',
 'mystery',
 'novels,',
 'we',
 'follow',
 'the',
 'journey',
 'of',
 'the',
 'protagonist',
 'who',
 'is',
 'would',
 'be',
 'trying',
 'to',
 'solve',
 'a',
 'crime.',
 'In',
 'thriller',
 'novels,',
 'we',
 'see',
 'the',
 'central',

##### (3) s, /s 추가

In [226]:
sentence_train.insert(0, '<s>')
sentence_train.append('</s>')

sentence_train

['<s>',
 'Suspense',
 'novels',
 'follow',
 'a',
 'plot',
 'that',
 'induces',
 'a',
 'unsettling',
 'sense',
 'of',
 'worry',
 'or',
 'anticipation',
 'in',
 'readers.',
 'This',
 'is',
 'done',
 'so',
 'by',
 'letting',
 'them',
 'in',
 'on',
 'details',
 'that',
 'the',
 "story's",
 'protagonist',
 'is',
 'often',
 'not',
 'weary',
 'of.',
 'Often,',
 'mystery',
 'or',
 'thriller',
 'novels',
 'fall',
 'under',
 'the',
 'suspense',
 'category',
 'and',
 'these',
 'three',
 'genre',
 'titles',
 'are',
 'also',
 'used',
 'interchangeably',
 'because',
 'of',
 'the',
 'blur',
 'lines.',
 'But',
 'the',
 'distinguishing',
 'factor',
 'between',
 'these',
 'three',
 'lies',
 'in',
 'the',
 'type',
 'of',
 'suspense',
 'that',
 'the',
 'story',
 'follows.',
 'In',
 'mystery',
 'novels,',
 'we',
 'follow',
 'the',
 'journey',
 'of',
 'the',
 'protagonist',
 'who',
 'is',
 'would',
 'be',
 'trying',
 'to',
 'solve',
 'a',
 'crime.',
 'In',
 'thriller',
 'novels,',
 'we',
 'see',
 'the',
 'c

##### 2) N-gram
- 순서에 맞게 수정

In [227]:
ngrams_train = [([sentence_train[i+j-NGRAM_SIZE] for j in range(NGRAM_SIZE)], sentence_train[i]) for i in range(NGRAM_SIZE, len(sentence_train))]
ngrams_train = ngrams_train[1:]
print(ngrams_train)

[(['Suspense', 'novels'], 'follow'), (['novels', 'follow'], 'a'), (['follow', 'a'], 'plot'), (['a', 'plot'], 'that'), (['plot', 'that'], 'induces'), (['that', 'induces'], 'a'), (['induces', 'a'], 'unsettling'), (['a', 'unsettling'], 'sense'), (['unsettling', 'sense'], 'of'), (['sense', 'of'], 'worry'), (['of', 'worry'], 'or'), (['worry', 'or'], 'anticipation'), (['or', 'anticipation'], 'in'), (['anticipation', 'in'], 'readers.'), (['in', 'readers.'], 'This'), (['readers.', 'This'], 'is'), (['This', 'is'], 'done'), (['is', 'done'], 'so'), (['done', 'so'], 'by'), (['so', 'by'], 'letting'), (['by', 'letting'], 'them'), (['letting', 'them'], 'in'), (['them', 'in'], 'on'), (['in', 'on'], 'details'), (['on', 'details'], 'that'), (['details', 'that'], 'the'), (['that', 'the'], "story's"), (['the', "story's"], 'protagonist'), (["story's", 'protagonist'], 'is'), (['protagonist', 'is'], 'often'), (['is', 'often'], 'not'), (['often', 'not'], 'weary'), (['not', 'weary'], 'of.'), (['weary', 'of.'],

##### 3) 단어 사전 만들기

In [228]:
vocab = set(sentence_train)
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {v:k for k, v in word_to_idx.items()}

In [229]:
word_to_idx

{'follows.': 0,
 'And': 1,
 'suspended': 2,
 'sense': 3,
 'be': 4,
 'dangerous': 5,
 'its': 6,
 'potentially': 7,
 'in': 8,
 'by': 9,
 'crime.': 10,
 'novels,': 11,
 '</s>': 12,
 'aware': 13,
 'these': 14,
 'under': 15,
 'details': 16,
 'novels': 17,
 'of.': 18,
 'distinguishing': 19,
 'lies': 20,
 'ever-present': 21,
 'also': 22,
 'This': 23,
 'readers': 24,
 'not': 25,
 'follow': 26,
 'that': 27,
 'story': 28,
 'mystery': 29,
 'characters': 30,
 'lines.': 31,
 'often': 32,
 'or': 33,
 'trying': 34,
 'protagonist': 35,
 'are': 36,
 'unsettling': 37,
 'thriller': 38,
 'so': 39,
 'solve': 40,
 'journey': 41,
 'readers.': 42,
 'three': 43,
 'see': 44,
 'titles': 45,
 'very': 46,
 'of': 47,
 'type': 48,
 'fall': 49,
 'worry': 50,
 'interchangeably': 51,
 'danger': 52,
 'But': 53,
 'factor': 54,
 'main': 55,
 'could': 56,
 'we': 57,
 'weary': 58,
 'suspense': 59,
 'genre': 60,
 'the': 61,
 'anticipation': 62,
 'central': 63,
 'who': 64,
 'because': 65,
 'risk.': 66,
 'an': 67,
 'plot': 68,

## Ngram pytorch module class 설정

##### 4) 모델 정의

In [230]:
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, EMBEDDING_DIM, NGRAM_SIZE):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim=EMBEDDING_DIM)
        self.linear1 = nn.Linear(NGRAM_SIZE * EMBEDDING_DIM, 128)
        self.linear2 = nn.Linear(128, vocab_size)
    def forward(self, inputs):
        # print(f'inputs : {inputs}') # tensor([51, 64])
        embeds = self.embeddings(inputs).view((1, -1)) # 행방향 크기를 1로 하고, 열방향에 나머지 원소들 다 때려넣어
        # print(f'embeds : {embeds}')
        """
        tensor([[-0.7379, -0.2725, -1.2734, -0.5576,  0.0750,  0.4588, -0.9168, -0.1681,
         -1.8120, -0.1912, -0.4644,  0.0171, -0.0437,  1.1693,  1.1770, -0.2649,
          1.0498,  0.4795, -0.7503,  1.0908]], grad_fn=<ViewBackward0>)
        """
        output = F.relu(self.linear1(embeds))
        output = self.linear2(output)
        log_probs = F.log_softmax(output, dim=1) # 열방향으로 Log Softmax
        # print(f'log_probs : {log_probs}')
        """
        tensor([[-4.7832, -4.7157, -4.7879, -4.5430, -4.4339, -4.5025, -4.2623, -4.0010,
         -4.7352, -4.6143, -4.3309, -4.6151, -4.3560, -4.4804, -4.7485, -4.5814,
         -4.6898, -4.2726, -4.9978, -4.6067, -4.1026, -4.1331, -4.5865, -4.7218,
         -4.7840, -4.6431, -4.7147, -4.6294, -4.7187, -4.2708, -4.6978, -4.4084,
         -4.7089, -4.2476, -4.4089, -4.6950, -4.6561, -4.9805, -4.7100, -4.6702,
         -4.6025, -4.3475, -4.6389, -4.8811, -4.6336, -4.9418, -4.8256, -5.0364,
         -4.3126, -4.4835, -4.3877, -4.8331, -4.5688, -4.8238, -4.7488, -4.7655,
         -4.7157, -4.4630, -4.4461, -4.5065, -4.6963, -5.0180, -4.6433, -4.2379,
         -4.5765, -4.7061, -4.6288, -4.7156, -4.8744, -4.7938, -4.5158, -4.5270,
         -4.6661, -4.4159, -4.7725, -4.5816, -4.4162, -4.6457, -4.4713, -4.6018,
         -4.6241, -4.4262, -4.5049, -4.2798, -4.6101, -4.8002, -4.4954, -4.9527,
         -4.6375, -4.5028, -4.7188, -4.0668, -4.7514, -4.6962, -4.8468, -4.4820,
         -4.5518]], grad_fn=<LogSoftmaxBackward0>)
        """
        return log_probs
    
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, NGRAM_SIZE)

In [231]:
print(model)

NGramLanguageModeler(
  (embeddings): Embedding(92, 10)
  (linear1): Linear(in_features=20, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=92, bias=True)
)


##### 5) Optimzier 정의

In [232]:
optimizer = optim.SGD(model.parameters(), lr=0.01)

##### 6) 모델 학습

In [233]:
losses = []
loss_function = nn.NLLLoss()

# (1) Epoch 순회
num_epochs = 100
for epoch in range(num_epochs + 1):
    # 1] 변수 초기화
    total_loss = 0
    for context, next_word in ngrams_train:
        # 2] train 데이터 전처리
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words into integer indices and wrap them in tensors)
        context_idxes = torch.tensor([word_to_idx[c] for c in context], dtype=torch.long)

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a new instance, you need to zero out the gradients from the old instance
        model.zero_grad()

        # 3] 모델 예측
        # Step 3. Run the forward pass, getting log probabilities over next words
        log_probs = model(context_idxes)

        # 4] Cost 계산
        # Step 4. Compute your loss function. (Again, Torch wants the target word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_idx[next_word]], dtype=torch.long))
        # print(torch.tensor([word_to_idx[next_word]], dtype=torch.long)) # tensor([61])

        # 5] 모델 역전파
        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)

# 6] 결과 출력
print(f'losses : {losses}')

print(model.embeddings.weight[word_to_idx["Suspense"]])

losses : [666.2148597240448, 625.5793209075928, 590.1961398124695, 557.350429058075, 524.9565241336823, 492.2175736427307, 458.98890471458435, 425.5077577829361, 392.1096041202545, 359.0969905257225, 326.6119393110275, 295.06555311381817, 264.8008646965027, 236.34968575835228, 210.02487459778786, 186.28239931166172, 165.10969195514917, 146.61354123055935, 130.61559999734163, 116.89307923614979, 105.17908681184053, 95.23790773749352, 86.81012922152877, 79.68799680098891, 73.66704379022121, 68.56312000378966, 64.22043144330382, 60.51112983375788, 57.37409371510148, 54.66105133853853, 52.31294694170356, 50.275880847126245, 48.484878888353705, 46.92233865708113, 45.53008865006268, 44.28667985647917, 43.18890555202961, 42.19733400084078, 41.30102359689772, 40.48098659515381, 39.735495262779295, 39.046260443516076, 38.43332285154611, 37.83463633712381, 37.30113116558641, 36.78732426278293, 36.32392826862633, 35.869585807435215, 35.44751384947449, 35.04990298859775, 34.6838352503255, 34.31866

- 테스트

In [234]:
context = ['Suspense', 'novels']

context

['Suspense', 'novels']

In [235]:
context_idxes = torch.tensor([word_to_idx[c] for c in context], dtype = torch.long)

context_idxes

tensor([91, 17])

In [236]:
log_probs = model(context_idxes)

log_probs

tensor([[-9.9061e+00, -1.0498e+01, -1.0758e+01, -9.2008e+00, -1.0155e+01,
         -1.3518e+01, -1.2190e+01, -9.6378e+00, -9.7001e+00, -1.2342e+01,
         -1.0333e+01, -9.7490e+00, -1.6533e+01, -1.4480e+01, -1.6123e+01,
         -1.0230e+01, -1.1962e+01, -6.5768e+00, -7.7794e+00, -1.2537e+01,
         -1.1348e+01, -1.1538e+01, -1.4145e+01, -1.0613e+01, -1.3330e+01,
         -1.7647e+01, -6.5814e-03, -8.7284e+00, -1.2222e+01, -1.6708e+01,
         -1.6745e+01, -1.5558e+01, -9.4164e+00, -1.2281e+01, -1.2313e+01,
         -1.7048e+01, -1.6297e+01, -1.2313e+01, -1.8562e+01, -1.2466e+01,
         -1.3149e+01, -1.6485e+01, -1.3789e+01, -1.1736e+01, -1.2244e+01,
         -1.5884e+01, -1.3463e+01, -1.2594e+01, -1.3223e+01, -5.9292e+00,
         -1.2614e+01, -1.1923e+01, -1.7659e+01, -1.4877e+01, -1.3528e+01,
         -1.4358e+01, -9.3623e+00, -1.3104e+01, -1.1925e+01, -1.3437e+01,
         -1.1851e+01, -8.4969e+00, -1.3920e+01, -1.2400e+01, -1.0834e+01,
         -9.9754e+00, -1.3599e+01, -1.

In [237]:
log_probs.sum()

tensor(-1146.7679, grad_fn=<SumBackward0>)

In [238]:
next_word_idx = log_probs.argmax(dim = 1)

next_word_idx

tensor([26])

In [240]:
next_word_idx.tolist()

[26]

In [241]:
next_word_idx.tolist()[0]

26

In [242]:
next_word = idx_to_word[next_word_idx.tolist()[0]]

next_word

'follow'

In [243]:
sentence_generate = ["<s>"]

In [244]:
sentence_generate.append(next_word)

sentence_generate

['<s>', 'follow']

In [245]:
ngrams_generate = sentence_generate[-NGRAM_SIZE:]

ngrams_generate

['<s>', 'follow']

In [246]:
word_to_idx["<s>"]

80

In [247]:
word_to_idx["Suspense"]

91

In [248]:
[word_to_idx["<s>"], word_to_idx["Suspense"]]

[80, 91]

In [249]:
temp_sentence = torch.tensor([word_to_idx["<s>"], word_to_idx["Suspense"]], dtype = torch.long)

temp_sentence

tensor([80, 91])

In [250]:
temp_sentence = torch.cat((temp_sentence, torch.tensor([0])))

temp_sentence

tensor([80, 91,  0])

In [251]:
temp_sentence = torch.cat((temp_sentence, torch.tensor([10], dtype = torch.long)))

temp_sentence

tensor([80, 91,  0, 10])

In [256]:
["<s>", 0]

['<s>', 0]

## 과제1: Generator 함수를 구현하세요.
 - 단어 하나를 입력으로 받아 문장을 자동으로 생성하는 def generator() 함수를 구현하면 됩니다.

In [252]:
def generate_sentence(start_word, seed=None, debug=False):
    # (1) 시드 초기화
    if seed is not None:
        import random
        random.seed(seed)
    # (2) 변수 초기화
    sentence_generate = ["<s>", start_word]
    context_generate = ["<s>", start_word]
    context_word = start_word

    i = 0
    # (3) 문장 생성
    while True:
        # 1] 현재 단어가 사전에 없는 경우 : 문장 생성 중단
        if context_word not in word_to_idx:
            break
        # 2] 다음 단어 생성
        context_ngram = torch.tensor([word_to_idx[s] for s in context_generate[-NGRAM_SIZE:]], dtype = torch.long)
        # print(context_ngram)
        next_log_probs = model(context_ngram)
        next_word_idx = next_log_probs.argmax(dim = 1)
        next_word = idx_to_word[next_word_idx.tolist()[0]]
        # 3] 다음 단어가 문장의 마지막인 경우 : 문장 생성 중단
        if next_word == "</s>":
            break
        # 4] 단어 전처리 (보편적으로 띄어쓰기를 하지 않는 경우를 제외하고는, 각 토큰은 띄어쓰기로 구분해서 생성)
        if context_word == "<s>":
            sentence_generate.append(next_word.title())
        elif context_word in ["`", "\"","'","("]:
            sentence_generate.append(next_word)
        elif next_word in ["'", ".", ",", ")", ":", ";", "?"]:
            sentence_generate.append(next_word)
        else:
            sentence_generate.append(" " + next_word)
        context_generate.append(next_word)
        context_word = next_word
        # 5] 결과 출력
        if debug:
            print(sentence_generate)
        i += 1
        if i == 100:
            break

    return "".join(sentence_generate)

In [253]:
generate_sentence("Suspense", debug = True)

['<s>', 'Suspense', ' who']
['<s>', 'Suspense', ' who', ' is']
['<s>', 'Suspense', ' who', ' is', ' would']
['<s>', 'Suspense', ' who', ' is', ' would', ' be']
['<s>', 'Suspense', ' who', ' is', ' would', ' be', ' trying']
['<s>', 'Suspense', ' who', ' is', ' would', ' be', ' trying', ' to']
['<s>', 'Suspense', ' who', ' is', ' would', ' be', ' trying', ' to', ' solve']
['<s>', 'Suspense', ' who', ' is', ' would', ' be', ' trying', ' to', ' solve', ' a']
['<s>', 'Suspense', ' who', ' is', ' would', ' be', ' trying', ' to', ' solve', ' a', ' crime.']
['<s>', 'Suspense', ' who', ' is', ' would', ' be', ' trying', ' to', ' solve', ' a', ' crime.', ' In']
['<s>', 'Suspense', ' who', ' is', ' would', ' be', ' trying', ' to', ' solve', ' a', ' crime.', ' In', ' thriller']
['<s>', 'Suspense', ' who', ' is', ' would', ' be', ' trying', ' to', ' solve', ' a', ' crime.', ' In', ' thriller', ' novels,']
['<s>', 'Suspense', ' who', ' is', ' would', ' be', ' trying', ' to', ' solve', ' a', ' crime.

'<s>Suspense who is would be trying to solve a crime. In thriller novels, we see the central characters or main protagonist being suspended in ever-present danger and risk. And in suspense novels, the plot takes its readers through an unsettling sense of worry and anticipation in readers by revealing potentially dangerous details that the very protagonist in the story could not be aware of.'

## 과제2: RNN을 이용해 생성모델을 강화해 보세요

## 과제3: Perplexity를 이용해 FCN모델과 RNN모델의 성능을 평가해보세요.